<a href="https://colab.research.google.com/github/alirempel/cap-comp215/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2D Circle Simulation
### Author: Alison R
### Date: 2024-02-05

ADD DESCRIPTION OF PROJECT HEREEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE

In [2]:
import matplotlib.pyplot as plt
import random
import math

### ASK ABOUT USING OLD STUFF

In [6]:
class Circle:
    """ Representation for the data and operations on a circle """

    def __init__(self, radius:float, centre:tuple):
      assert len(centre) == 2
      self.radius = radius
      self.centre = centre

    def area(self):
        """ Return the area of this circle """
        return math.pi * self.radius**2

    def move(self, offset:tuple):
        """ Move this circle by given (x,y) offset """
        self.centre = (self.centre[0] + offset[0], self.centre[1] + offset[1])

    def distance(self, other):
        """ Return the distance between this circle's centre and the other one """
        return math.dist(self.centre, other.centre)

    def intersects(self, other):
        """ Return True iff this circle intersects the other one """
        return self.radius + other.radius > self.distance(other)

In [7]:
# Test Circle class
def test_circle():
  c1 = Circle(3,(2,4))
  c2 = Circle(2,(5,8))
  assert c1 != c2 and c1.radius != c2.radius and c1.centre != c2.centre

  assert c1.area() == 9 * math.pi
  assert c1.distance(c2) == c2.distance(c1)
  assert c1.distance(c2) == 5.0
  assert c1.intersects(c2) == False
  c1.move((1,1))
  assert c1.intersects(c2) == True

  print('tests passed!!')

test_circle()

tests passed!!
